## (*) Prerequisite

### Install required packages
1. Go to the root directory and install the required packages:
```
pip install -r requirements.txt
```
2. set your jupyter notebook kernel correctly to the environment with all required packages.


### Set up environment

In [1]:
import sys
sys.path.append('../')

## (A) Generate PDF Report

This demo is to use XAI report formatter (a.k.a `xai.formatter.report`) to generate report content.
### Step 1: Create Report object
Create a report object

In [2]:
from xai.formatter.report import Report
report = Report()
print(report)

### Step 2: Create section and add content to report 
Content refer a content block inside report. XAI library group certain popular content into business group, some examples includes but not limited to:
- Dataset distribution
- Feature value distribution (based on feature type: categorical, numeric, text, etc.)
- Evaluation result summary
- Feature importance
- Learning curve history
- ...

These contents are created by functions start with `add_xx` in section, then each contents is added to report section that created using 
- `create_cover`: cover section 
- `create_numbering_section`: section with top level numbering (e.g. 1, 2, 3, ..)
- `create_numbering_sub_section`: section with 2nd level numbering (e.g. 1.1, 1.2, 1.3, 2.1, ..) 
- `create_numbering_sub_sub_section`: section with third level numbering (e.g. 1.1.1, 1.1.2, 2.1.1, 2.1.2, ..) 

The cover page has a default design with the following information:
//todo:

The content table is enabled by default, call `has_content_table` and set indicator as `False` to disable
The content table will be in the follow format:
> **Content Table**
>
>**1 Section Title**
>
>... 1.1 SubSection Title
>
>*...... 1.1.1 SubSubSection Title*
>
>**2 Section Title**
>
>... 2.1 SubSection Title
>
>*...... 2.1.1 SubSubSection Title*
>
>*...... 2.1.2 SubSubSection Title*

Below is a sample to create different level of sections and add content(s) to it.

In [3]:
### Create Cover Section
section = report.create_cover(title='Report Summary (created)')
section.add_paragraph(text="This is summary Info")

### Create Numbering Root Section
section = report.create_numbering_section(title='Section Title')
section.add_paragraph(text="This is Section Info")

### Create Numbering Subsection
section = report.create_numbering_sub_section(title='Sub-Section Info')
section.add_paragraph(text="This is Sub-Section Info")

### Create Numbering Sub-Subsection
section = report.create_numbering_sub_sub_section(title='Sub-Sub-Section Info')
section.add_paragraph(text="This is Sub-Sub_Section Info")

### Create Paragraph Section
section = report.create_paragraph_section(title='Paragraph Title')
section.add_paragraph(text="This is Paragraph Info")

### -- alternative --
### Create Numbering Root Section 2
report.create_numbering_section(title='Section2 Title').add_paragraph(text="This is Section2 Info")

### Create Numbering Sub-Subsection
report.create_numbering_sub_sub_section(
    title='Sub-Sub-Section Info - Section 2').add_paragraph(text="This is Sub-Sub_Section Info in Section 2")
### Create Paragraph Section
report.create_paragraph_section(title='Paragraph Title II').add_paragraph(text="This is Paragraph Info II")
    

### Step 3: Generate the report

Finally, call `generate` with `writer` object to compile the report.

Below is sample to generate report in PDF format:

In [4]:
from xai.formatter.portable_document import PdfWriter
report.generate(writer=PdfWriter(path='./', name='Sample Report (notebook)'))

***

## (B) Contents Examples
The following section introduces a default design of report, which includes the following parts and each will be shown with a sample dummy data:
- **0. Cover page** (display at the beginning of the report but generated at the end of whole process)
- **1. Data Analysis**
- **2. Feature Analysis**
- **3. Training Analysis**
    - *3.1 Hyperparameter Tuning*
    - *3.2 Learning Curve (for deep learning)*
- **4. Evaluation Analysis**
- **5. Recommendataion Analysis (TBE)**

First create a report object :)

In [9]:
import json
report = Report()

### 1. Data Analysis 
The section includes all the information for data analysis. By default, it should have the following information:
- Class distribution (for classification problem)
- Data field attribute
- Data field value distribution
- Missing value check

Let's add paragraph to begin with.

In [12]:
### Data - Paragraph Section
report.create_paragraph_section(title='Data Analysis').add_paragraph(text="This section explains details on the data.")

### 1.1 Class Distribution
This section displays the distribution of each class given a label. It's for classification problem. The component is created with `component_dataset_distribution`. 

The data passed in should be a list of tuple. The first item is the label name, the second item is a dict which represents a distribution under the label, and each key-value pair is a class label and the corresponding quantity under the label. This data structure is compatible with the `data_analyzer` in `xai.data_explorer` package. If the data is generated from xai package, it can be directly passed into the function.

A sample is shown as follow:

In [8]:
"""Sample: Dataset distribution
"""
with open('./sample_data/data_distribution.json','r') as f:
    data_dist = json.load(f)
    
data_distributions = []
for k, v in data_dist.items():
    data_distributions.append((k,v))
print(data_distributions)
    
comp = report.component_dataset_distribution(*data_distributions)
report.add_content_as_subsection_to_report_buffer(title='Data Distribution', content=comp)

[('Category I', {'OTHER': 5651, 'OFFENSE': 2890}), ('Category II', {'OTHER': 5651, 'INSULT': 976, 'PROFANITY': 119, 'ABUSE': 1795})]


### 1.2 Data Field Attribute
This section displays the attribute for all data fields. The component is created with `component_data_attributes`. 

The data passed in should be a dict. For each item, the key is the field name, the value is a second-level dict with items of which key is the attribute name and value is the attribute value.

A sample is shown as follow:

In [9]:
"""Sample: Data Field Attribute
"""
with open('./sample_data/data_attribute.json','r') as f:
    data_attribute = json.load(f)
print(data_attribute)  

comp = report.component_data_attributes(data_attribute=data_attribute)
report.add_content_as_subsection_to_report_buffer(title='Data Field Attribute', content=comp)

{'id': {'type': 'Key', 'used': False, 'structured': 'attribute'}, 'subject': {'type': 'Text', 'used': True, 'structured': 'attribute'}, 'description': {'type': 'Text', 'used': True, 'structured': 'attribute'}, 'textload': {'type': 'Key', 'used': False, 'structured': 'attribute'}, 'label_0': {'type': 'Label', 'used': True, 'structured': 'attribute'}, 'label_1': {'type': 'Label', 'used': True, 'structured': 'attribute'}, 'has_emoji': {'type': 'Nominal', 'used': False, 'structured': 'attribute'}}


### 1.3 Missing Value Check
This section displays the missing value checking. The component is created with component_data_missing_value.

The function takes in two inputs: one is `missing_count` which is a dict with items of which key is field name and value is the count or ratio of missing attribute. 

A sample is shown as follow:

In [11]:
"""
Sample: Missing value
"""
with open('./sample_data/missing_value.json','r') as f:
    missing_value = json.load(f)
    
missing_count = missing_value["missing_count"]
total_count = missing_value["total_count"]
print('missing_count',missing_count)
print('total_count',total_count)

section = report.create_numbering_sub_section(title='Missing Value Checking')
section.add_data_missing_value(missing_count=missing_count, total_count=total_count)

missing_count {'description': 799, 'subject': 140}
total_count {'id': 8541, 'subject': 8541, 'description': 8541, 'label_0': 8541, 'label_1': 8541, 'textload': 8541, 'has_emoji': 8541}


### 1.4 Data Field Distribution

This section displays value distribution for data fields. `xai.data_explorer` package analyzes the distribution for the following types:
- Categorical
- Numerical
- Free Text
- Datetime


In [11]:
"""Sample: Data Field Distribution
"""
section_description = 'This section displays distribution for categorical fields, numerical fields and text fields.'
comp = report.component_paragraph_in_html(section_description)
report.add_content_as_subsection_to_report_buffer(title='Data Field Distribution', content=comp)

#### 1.4.1 Categorical Field Distribution
For categorical field, the value frequency under each label will be shown. 
See details about `xai.data_explorer.categorical_analyzer`. 

In [12]:
"""Sample: Categorical field distribution
"""    
report.add_content_as_subsubsection_to_report_buffer(title='Categorical Field Distribution')

with open('./sample_data/categorical_data.json','r') as f:
    categorical_data = json.load(f)

print(categorical_data)
    
for field_name, field_distribution in categorical_data.items():
    comp = report.component_categorical_field_distribution(field_name=field_name, field_distribution=field_distribution)
    report.add_content_to_report_buffer(title='Distribution for %s' % field_name, content=comp)

{'has_emoji': {'OTHER': {'false': 4902, 'true': 749}, 'OFFENSE': {'false': 2544, 'true': 346}, 'all': {'false': 7446, 'true': 1095}}}


#### 1.4.2 Numerical Field Distribution
For numerical fields, the value distribution will shown as histogram and KDE curve. Main statistics (e.g. max, min, mean, median) will also be shown.
See details about `xai.data_explorer.numerical_analyzer`. 

In [13]:
"""Sample: Numerical field distribution
"""    
report.add_content_as_subsubsection_to_report_buffer(title='Numerical Field Distribution')

with open('./sample_data/numerical_data.json','r') as f:
    numerical_data = json.load(f)

print('SAMPLE DATA FORMAT')
print('==================')
for key,value in numerical_data.items():
    print('Field name:',key)
    for class_key, class_stats in value.items():
        print(' - Class name:', class_key)
        for stats_key, stats_value in class_stats.items():
            print('   * %s:'%stats_key, type(stats_value), stats_value[:min(3,len(stats_value))] if type(stats_value)==list else stats_value)
        break
        
for field_name, field_distribution in numerical_data.items():
    comp = report.component_numeric_field_distribution(field_name=field_name, field_distribution=field_distribution)
    report.add_content_to_report_buffer(title='Distribution for %s' % field_name, content=comp)


SAMPLE DATA FORMAT
Field name: subject_textlength
 - Class name: all
   * x_limit: <class 'list'> [2.0, 162.0]
   * kde: <class 'list'> [[-17.08751583869025, 2.0041936959240893e-05], [-15.085747841949035, 5.410551190083066e-05], [-13.083979845207818, 0.00013309594681131046]]
   * histogram: <class 'list'> [[2.0, 0.0, 5.517241379310345, 0.01460001231678778], [7.517241379310345, 0.0, 5.517241379310345, 0.013573100135484665], [13.03448275862069, 0.0, 5.517241379310345, 0.010938847148663628]]
   * total_num: <class 'int'> 8541
   * max: <class 'float'> 444.0
   * min: <class 'float'> 2.0
   * mean: <class 'float'> 61.33426999180424
   * median: <class 'float'> 48.0
   * perc_10: <class 'float'> 10.0
   * perc_90: <class 'float'> 127.0


#### 1.4.3 Text Field Distribution
For text fields, a woldcloud will be shown with a size weigthed on the TF-IDF. Besides, XAI has some pre-defined text patterns, such as `Email`, `URL`, `Datetime`. If the text field is analyzed with XAI Data Explorer package, the percentage of documents contains each pattern will also be analyzed and displayed.
See details about `xai.data_explorer.text_analyzer`. 

In [14]:
"""Sample: Text field distribution
"""    
report.add_content_as_subsubsection_to_report_buffer(title='Text Field Distribution')

with open('./sample_data/text_data.json','r') as f:
    text_data = json.load(f)

print('SAMPLE DATA FORMAT')
print('==================')
for key,value in text_data.items():
    print('Field name:',key)
    for class_key, class_stats in value.items():
        print(' - Class name:', class_key)
        for stats_key, stats_value in class_stats.items():
            print('   * %s:'%stats_key, type(stats_value), stats_value[:min(3,len(stats_value))] if type(stats_value)==list else stats_value)
        break
        

for field_name, field_distribution in text_data.items():
    comp = report.component_text_field_distribution(field_name=field_name, field_distribution=field_distribution)
    report.add_content_to_report_buffer(title='Distribution for %s' % field_name, content=comp)

SAMPLE DATA FORMAT
Field name: description
 - Class name: all
   * tfidf: <class 'list'> [['lbr', 2398.6085830305574], ['nan', 1893.0486717137182], ['dass', 883.4445210485602]]
   * placeholder: <class 'dict'> {'DATE': 0.0071420208406509774, 'NUMBER': 0.07095187917105725, 'EMAIL': 0.08394801545486477, 'TIME': 0.0012879053974944386}


#### 1.4.4 DateTime Field Distribution
For datetime fields, count on months/years will be show in an barchart. 
See details about `xai.data_explorer.datetime_analyzer`. 

In [15]:
"""Sample: Datetime field distribution
"""    
report.add_content_as_subsubsection_to_report_buffer(title='Datetime Field Distribution')

with open('./sample_data/datetime_data.json','r') as f:
    datetime_data = json.load(f)
    
print(datetime_data)

for field_name, field_distribution in datetime_data.items():
    comp = report.component_datetime_field_distribution(field_name=field_name, field_distribution=field_distribution)
    report.add_content_to_report_buffer(title='Distribution for %s' % field_name, content=comp)

{'POSTINGDATE': {'all': {'2017': {'11': 2360, '12': 1796, '9': 2507, '3': 1409, '2': 1481, '6': 1439, '1': 3962, '5': 1369, '4': 1229, '7': 1582, '8': 2277, '10': 2329}, '2018': {'1': 1688, '2': 1138, '3': 483, '4': 10}, '2015': {'4': 2123, '2': 1374, '6': 1593, '10': 1247, '11': 1058, '1': 4094, '12': 1592, '3': 1526, '7': 1420, '5': 1035, '8': 1133, '9': 1209}, '2016': {'12': 2007, '2': 1027, '1': 4497, '3': 1297, '4': 1342, '11': 1435, '7': 1532, '8': 1220, '6': 1566, '10': 1522, '9': 1544, '5': 1047}, '2014': {'12': 1506, '11': 631, '10': 530, '7': 464, '8': 260, '9': 473, '3': 352, '1': 995, '6': 432, '5': 331, '4': 553, '2': 256}, '2013': {'6': 481, '3': 331, '11': 250, '9': 324, '4': 434, '12': 343, '10': 371, '5': 294, '7': 450, '8': 281, '1': 881, '2': 172}, '2011': {'6': 1, '10': 4, '12': 3, '3': 2, '2': 2, '4': 3, '7': 2, '9': 1, '11': 1}, '2012': {'12': 302, '4': 15, '3': 4, '11': 215, '10': 254, '9': 75, '8': 41, '7': 46, '1': 11, '6': 7, '5': 6, '2': 1}, '2009': {'1': 2, 

### 2. Feature Analysis
This section shows analysis for feature importance. It shows as a barchart and feature importance above a thresdhold as a table. A sample is shown as below:

In [16]:
report.add_content_as_section_to_report_buffer(title='Feature Analysis')

with open('./sample_data/feature_importance.json', 'r') as f:
    feature_importance = json.load(f)
    
print('SAMPLE DATA FOR FEATURE IMPORTANCE')
print('==================================')
print(type(feature_importance),feature_importance[:10])

comp = report.component_feature_importance(importance_ranking=feature_importance, importance_threshold=0.005)
report.add_content_as_subsection_to_report_buffer(title='Feature Importance', content=comp)


SAMPLE DATA FOR FEATURE IMPORTANCE
<class 'list'> [[0.0457, 'Months_to_end'], [0.0437, 'Nth_Month'], [0.0281, 'Curr_Calendar_Month'], [0.0275, 'IC_0/ContactPostalCode-nil'], [0.0185, 'Agg_Curr_Num_IA_last45days'], [0.0175, 'IC_0/ContactPostalCode-94010'], [0.0169, 'IC_0/ContactPostalCode-95125'], [0.0169, 'Agg_BF_Num_IA_last360days'], [0.0165, 'IC_0/GenderCode-1'], [0.0138, 'Agg_BF_Num_IA_0/InteractionType_Ticket_Purchase_last360days']]


### 3. Training Analysis
This section shows training analysis, and presents information while training process.

In [17]:
report.add_content_as_section_to_report_buffer(title='Training Analysis')

### 3.1 Hyperparameter Tuning
The section shows model metric monitoring while tuning hyperparameters.

In [18]:
with open('./sample_data/hyperparameter_tuning.json', 'r') as f:
    hyperparameter_tuning = json.load(f)

print('search_space:',hyperparameter_tuning['search_space'])
print('best_idx:',hyperparameter_tuning['best_idx'])
print('history [first 2 samples]:',
      {k:hyperparameter_tuning['history'][k] for k in list(hyperparameter_tuning['history'].keys())[:2]})
print('benchmark_metric:',hyperparameter_tuning['benchmark_metric'])
print('benchmark_threshold:',hyperparameter_tuning['benchmark_threshold'])
print('non_hyperopt_score:',hyperparameter_tuning['non_hyperopt_score'])

comp = report.component_hyperparameter_tuning(history=hyperparameter_tuning['history'],
                                              best_idx=hyperparameter_tuning['best_idx'],
                                              search_space=hyperparameter_tuning['search_space'],
                                              benchmark_metric=hyperparameter_tuning['benchmark_metric'],
                                              benchmark_threshold=hyperparameter_tuning['benchmark_threshold'],
                                              non_hyperopt_score=hyperparameter_tuning['non_hyperopt_score'])

report.add_content_as_subsection_to_report_buffer(title='Hyperparameter Tuning', content=comp)

search_space: {'NumTrees': '(20, 500)', 'RandomState': '(200, 1000)', 'MaxFeature': 'auto', 'MinSplit': 2, 'MaxDepth': ['None', 4, 5, 6]}
best_idx: 3
history [first 2 samples]: {'0': {'params': {'MaxDepth': 6, 'MaxFeature': 'auto', 'MinSampleSplit': 2, 'NumTrees': 448.0, 'RandomState': 685.0}, 'val_scores': {'accuracy': 0.075, 'f1': 0.07493575942773803, 'precision': 0.4217391304347826, 'recall': 0.4342105263157895, 'auc': 0.4009502923976608}}, '1': {'params': {'MaxDepth': 6, 'MaxFeature': 'auto', 'MinSampleSplit': 2, 'NumTrees': 351.0, 'RandomState': 351.0}, 'val_scores': {'accuracy': 0.0875, 'f1': 0.08748415771107138, 'precision': 0.44510335479498475, 'recall': 0.44078947368421056, 'auc': 0.40570175438596495}}}
benchmark_metric: accuracy
benchmark_threshold: 0.8
non_hyperopt_score: 0.8166666666666667


### 3.2 Learning Curve (for deep learning)
The section shows model metric monitoring while training (a.k.a learning curve).

In [19]:
with open('./sample_data/learning_curve.json', 'r') as f:
    learning_curve = json.load(f)

print('best_idx:',learning_curve['best_idx'])
print('history [first 2 samples]:',
      {k:learning_curve['history'][k] for k in list(learning_curve['history'].keys())[:2]})
print('benchmark_metric:',learning_curve['benchmark_metric'])
print('benchmark_threshold:',learning_curve['benchmark_threshold'])
print('training_params:',learning_curve['training_params'])

comp = report.component_learning_curve(history=learning_curve['history'],
                                       best_idx=learning_curve['best_idx'],
                                       benchmark_metric=learning_curve['benchmark_metric'],
                                       benchmark_threshold=learning_curve['benchmark_threshold'],
                                       training_params=learning_curve['training_params'])

report.add_content_as_subsection_to_report_buffer(title='Deep Learning Training', content=comp)

best_idx: 0
history [first 2 samples]: {'0': {'val_scores': {'loss': 0.769659698009491, 'accuracy': 0.9515625238418579, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'auc': 0.5082619190216064}}, '1': {'val_scores': {'loss': 0.7648007273674011, 'accuracy': 0.948437511920929, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'auc': 0.4868066608905792}}}
benchmark_metric: accuracy
benchmark_threshold: 0.9
training_params: {'Sequence time step': 15, 'Sequence feature length': 125, 'Attribute feature length': 294, 'RNN layer number': 2, 'RNN keep probability': 0.8, 'DNN layer number': 1, 'DNN regularizer': 'L2 (scale: 0.06)'}


### 4. Evaluation Analysis
The section provides analysis on evaluation results. It is divided into 2 types: binary classification and multi-class classification. For each case, the following components are displayed:
 - numeric metrics
 - confusion matrix
 - confidence distribution
 - reliability diagram (binary classification only)

### 4.1 Binary Classification 

In [20]:
report.add_content_as_section_to_report_buffer(title='Binary Classification Evaluation Analysis')

#### 4.1.1 Numeric Metric

In [21]:
with open('./sample_data/binary_evaluation_results.json', 'r') as f:
    eval_result = json.load(f)

splits = eval_result.keys()
numeric_metrics_list = []

for split in splits:
    label_eval_result = eval_result[split]
    del(label_eval_result['vis_result'])
    numeric_metrics_list.append((split, label_eval_result))

print(numeric_metrics_list)

comp = report.component_binary_class_evaluation_metric_results(*numeric_metrics_list,
                                                             notes='The section shows general results like accuracy, precision, recall.')
report.add_content_as_subsection_to_report_buffer(title='Overall Result',content=comp)

[('train', {'accuracy': 0.9627912640571594, 'precision': 0.02666666731238365, 'recall': 0.008547008968889713, 'f1': 0.012944985181093216, 'auc': 0.4609035849571228, 'CM': [[7890, 73], [232, 2]]}), ('validation', {'accuracy': 0.9576036930084229, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'auc': 0.4362090826034546, 'CM': [[1039, 14], [32, 0]]}), ('test', {'accuracy': 0.962892472743988, 'precision': 0.10000000149011612, 'recall': 0.032258063554763794, 'f1': 0.04878048971295357, 'auc': 0.4682953953742981, 'CM': [[1011, 9], [30, 1]]})]


#### 4.1.2 Confusion Matrix

In [22]:
with open('./sample_data/binary_evaluation_results.json', 'r') as f:
    eval_result = json.load(f)

splits = eval_result.keys()
confusion_matrix_list = []

for split in splits:
    label_eval_result = eval_result[split]
    confusion_matrix_list.append((split, {"labels": ["Negative", "Positive"], "values": label_eval_result["CM"]}))

print(confusion_matrix_list)

comp = report.component_confusion_matrix(*confusion_matrix_list)
report.add_content_as_subsection_to_report_buffer(title='Confusion Matrix', content=comp)

[('train', {'labels': ['Negative', 'Positive'], 'values': [[7890, 73], [232, 2]]}), ('validation', {'labels': ['Negative', 'Positive'], 'values': [[1039, 14], [32, 0]]}), ('test', {'labels': ['Negative', 'Positive'], 'values': [[1011, 9], [30, 1]]})]


#### 4.1.3 Confidence Distribution

In [23]:
with open('./sample_data/binary_evaluation_results.json', 'r') as f:
    eval_result = json.load(f)

splits = eval_result.keys()
probability_plot_list = []

for split in splits:
    vis_result = eval_result[split]['vis_result']
    probability_plot_list.append((split, vis_result))
    print('Split name:',split)
    for key, value in vis_result.items():
        print(' - %s:'%key, type(value), value[:3])
    
comp = report.component_binary_class_confidence_distribution(*probability_plot_list)
report.add_content_as_subsection_to_report_buffer(title='Confidence Distribution', content=comp)

Split name: train
 - probability: <class 'list'> [[0.3770688772201538, 0.6229311227798462], [0.41023093461990356, 0.5897691249847412], [0.4822709560394287, 0.5177291035652161]]
 - gt: <class 'list'> [0, 0, 0]
Split name: validation
 - probability: <class 'list'> [[0.44429945945739746, 0.5557004809379578], [0.5415756702423096, 0.45842432975769043], [0.46190792322158813, 0.5380920171737671]]
 - gt: <class 'list'> [0, 0, 0]
Split name: test
 - probability: <class 'list'> [[0.4792706072330475, 0.5207294225692749], [0.541035532951355, 0.4589644968509674], [0.47570356726646423, 0.5242964029312134]]
 - gt: <class 'list'> [0, 0, 0]


#### 4.1.4 Reliability Digram

In [24]:
with open('./sample_data/binary_evaluation_results.json', 'r') as f:
    eval_result = json.load(f)

splits = eval_result.keys()
probability_plot_list = []

for split in splits:
    label_eval_result = eval_result[split]['vis_result']
    probability_plot_list.append((split, vis_result))
    print('Split name:',split)
    for key, value in vis_result.items():
        print(' - %s:'%key, type(value), value[:3])

comp = report.component_binary_class_reliability_diagram(*probability_plot_list)
report.add_content_as_subsection_to_report_buffer(title='Reliability Diagram', content=comp)

Split name: train
 - probability: <class 'list'> [[0.4792706072330475, 0.5207294225692749], [0.541035532951355, 0.4589644968509674], [0.47570356726646423, 0.5242964029312134]]
 - gt: <class 'list'> [0, 0, 0]
Split name: validation
 - probability: <class 'list'> [[0.4792706072330475, 0.5207294225692749], [0.541035532951355, 0.4589644968509674], [0.47570356726646423, 0.5242964029312134]]
 - gt: <class 'list'> [0, 0, 0]
Split name: test
 - probability: <class 'list'> [[0.4792706072330475, 0.5207294225692749], [0.541035532951355, 0.4589644968509674], [0.47570356726646423, 0.5242964029312134]]
 - gt: <class 'list'> [0, 0, 0]


### 4.2 Multi-class Classification 

In [25]:
report.add_content_as_section_to_report_buffer(title='Multi-class Classification Evaluation Analysis')

with open('./sample_data/multi_evaluation_results.json', 'r') as f:
    eval_result = json.load(f)

label_key = 'label_1'
label_eval_result = eval_result[label_key]
vis_result = label_eval_result['vis_result']
del (label_eval_result['vis_result'])

#### 4.2.1 Numeric Metric

In [26]:
"""Sample: Numeric Metrics
"""
print(label_eval_result)

comp = report.component_multi_class_evaluation_metric_results(('', label_eval_result),
                                                              notes='The section shows general results like accuracy, precision, recall.')
report.add_content_as_subsection_to_report_buffer(title='Overall Result', content=comp)

{'f1_score': {'class': {'OTHER': 0.8106060606060606, 'PROFANITY': 0.0, 'ABUSE': 0.43083003952569177, 'INSULT': 0.03529411764705882}, 'average': 0.6306263638637892}, 'recall': {'class': {'OTHER': 0.9214208826695371, 'PROFANITY': 0.0, 'ABUSE': 0.3707482993197279, 'INSULT': 0.018633540372670808}, 'average': 0.6894586894586895}, 'precision': {'class': {'OTHER': 0.7235841081994928, 'PROFANITY': 0.0, 'ABUSE': 0.5141509433962265, 'INSULT': 0.3333333333333333}, 'average': 0.6246700003863861}, 'CM': {'labels': ['OTHER', 'PROFANITY', 'ABUSE', 'INSULT'], 'values': [[856, 0, 67, 6], [16, 0, 4, 0], [185, 0, 109, 0], [126, 0, 32, 3]]}}


#### 4.2.2 Confusion Matrix

In [27]:
"""Sample: Confusion Matrix
"""
print(label_eval_result['CM'])

comp = report.component_confusion_matrix(('', label_eval_result['CM']))
report.add_content_as_subsection_to_report_buffer(title='Confusion Matrix', content=comp)

{'labels': ['OTHER', 'PROFANITY', 'ABUSE', 'INSULT'], 'values': [[856, 0, 67, 6], [16, 0, 4, 0], [185, 0, 109, 0], [126, 0, 32, 3]]}


#### 4.2.3 Confidence Distribution

In [28]:
"""Sample: Probability Plot
"""
for class_name, class_value in vis_result.items():
    print('Class:',class_name)
    for key, value in class_value.items():
        print(' - %s:'%key, type(value), value[:4])
        
comp = report.component_multi_class_confidence_distribution(('', vis_result))
report.add_content_as_subsection_to_report_buffer(title='Confidence Distribution', content=comp)

Class: OTHER
 - gt: <class 'list'> ['OTHER', 'OTHER', 'INSULT', 'OTHER']
 - probability: <class 'list'> [0.8916968852945533, 0.6004393436260155, 0.6709159285290026, 0.8776989624158892]
Class: PROFANITY
 - gt: <class 'list'> []
 - probability: <class 'list'> []
Class: ABUSE
 - gt: <class 'list'> ['ABUSE', 'INSULT', 'OTHER', 'INSULT']
 - probability: <class 'list'> [0.3997915305239952, 0.4321055255119627, 0.4272960507553194, 0.9063218066182464]
Class: INSULT
 - gt: <class 'list'> ['INSULT', 'OTHER', 'OTHER', 'OTHER']
 - probability: <class 'list'> [0.4748126330295362, 0.4603932866512729, 0.4276429140622332, 0.3630582608615315]


### 5. Recommendataion Analysis (TBE)

### 0. Cover Page
The cover page contains following default information, but need to be called at the end of pipeline:
- Summary Notes
- Model Information
- Timing
- Data Summary
- Evaluation Result

#### Summary Note
It should be created by `component_paragraph_in_html` and contains the summary text in paragraph and display at the begining of the report.

In [29]:
summary_notes_text = '''This report serves as a demo for the XAI library. 
The data used to generate this report is not from any specific use case.
Sorry for any inconsistency in the report data. 
'''
summary_notes_comp = report.component_paragraph_in_html(summary_notes_text)

#### Model Information
The component is created by `component_model_info_summary`. It is designed to display the basid model information, such as `model_id`, `model_version` or any information specific to this training. 

The data should be passed in as a list of tuple, each tuple will be a key-value pair. If no data is provided to the component, `usecase_name`, `version`, `usecase_team` will be displayed as a default information. A sample is shown as follows:

In [30]:
model_info = [('Model ID','12345678'),
              ('Model Version','v6'),
              ('Scenario ID','111222333444555'),
              ('Notes','This model is created as a beta version.')
             ]
model_info_comp = report.component_model_info_summary(model_info=model_info)

#### Timing
The component is created by `component_training_timing`. It is designed to display the time spend at each stage of the training process, such as `data processing`, `feature extraction`, `training model` and `evaluation`. 

The data should be passed in as a list of tuple, each tuple will be a pair of stage name and time spend in seconds. A sample is shown as follows:

In [31]:
timing = [('Data Preprocessing', 1000),
          ('Feature Engineering', 10000),
          ('Training', 200200),
          ('Evaluation', 30303)]
timing_comp = report.component_training_timing(timing=timing)

#### Data Summary
The component is created by `component_dataset_summary`. It is designed to display the quantity of samples used in each stage of training (a.k.a training, validation, testing). It can also be used to display the distribution among different classes.

The data should be passed in as a list of tuple, each tuple will be a pair of dataset name and quantity. A sample is shown as follows:

In [32]:
data_summary = [('training', 10000), 
                ('validation', 2000), 
                ('testing', 1000)]
data_summary_comp = report.component_dataset_summary(data_summary=data_summary)

#### Evaluation Result Summary
The component is created by `component_evaluation_result_summary`. It is designed to display the summary on evaluation result in a high-level. Only numeric metrics will be displayed, anything involves a list won't be shown in summary. For example, for multi-class classification, only the average metrics will be shown, also metric such like ROC and confusion matrix won't be shown here.

The data should be passed in as a list of dict, each dict includes evaluation on one objective. For example, in a multi-label classication or multi-task machine learning problem, each dict will only regards on one label or one task. The function takes in as many as dict and display each of them in sequence.

For each dict, each key-value pair should about a metric or an attribute. 
 - If the value is a number or a str, the key-value pair will be displayed
 - If the value is a list, it will be ignored in display. 
 - If the value is dict, the function will search the keyword `average` to display the average metric; otherwise, search the keyword `class` and calculate a macro average if it's a list of float. For other cases, the function will ignore this item.
 
A sample is shown as follows:

In [33]:
import json
with open('./sample_data/evaluation_result_summary.json','r') as f:
    evaluation_result_data = json.load(f)
print(evaluation_result_data)

evaluation_result_comp = report.component_evaluation_result_summary(*evaluation_result_data)

[{'field': 'Category I', 'labels': 'OTHER, OFFENSE', 'f1_score': {'class': {'OTHER': 0.7871690427698573, 'OFFENSE': 0.504739336492891}, 'average': 0.6916176820280062}, 'recall': {'class': {'OTHER': 0.8320775026910656, 'OFFENSE': 0.44842105263157894}, 'average': 0.7022792022792023}, 'precision': {'class': {'OTHER': 0.7468599033816425, 'OFFENSE': 0.5772357723577236}, 'average': 0.6894728220167127}}, {'field': 'Category II', 'labels': 'OTHER, PROFANITY, ABUSE, INSULT', 'f1_score': {'class': {'OTHER': 0.8106060606060606, 'PROFANITY': 0.0, 'ABUSE': 0.43083003952569177, 'INSULT': 0.03529411764705882}, 'average': 0.6306263638637892}, 'recall': {'class': {'OTHER': 0.9214208826695371, 'PROFANITY': 0.0, 'ABUSE': 0.3707482993197279, 'INSULT': 0.018633540372670808}, 'average': 0.6894586894586895}, 'precision': {'class': {'OTHER': 0.7235841081994928, 'PROFANITY': 0.0, 'ABUSE': 0.5141509433962265, 'INSULT': 0.3333333333333333}, 'average': 0.6246700003863861}}]


After all the component needed to be in the cover is ready, `generate_cover_page` can be called to generate the cover. The program will put all the components at the beginning of the report buffer and add a content table automatically at the end of the cover page based on the components added before.

In [34]:
report.generate_cover_page(summary_notes=summary_notes_comp,
                           model_info=model_info_comp,
                           timing=timing_comp,
                           data_summary=data_summary_comp,
                           evaluation_result=evaluation_result_comp
                           )

In [35]:
report.output_report('./',report_name='training_report')

../xai/graphs/graph_generator.py:38: RuntimeWarning: invalid value encountered in double_scalars
  ave_acc = np.sum(accuracy[condition]) / sample_num
/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
